In [2]:
import os
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers.legacy import SGD  # Import the legacy SGD optimizer
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from google.colab import drive

# Mount Google Drive to '/content/drive'
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
# Define the paths to your train and test directories
train_dir = '/content/drive/MyDrive/kidney_data/train'  # Replace with your actual train directory path
test_dir = '/content/drive/MyDrive/kidney_data/train'    # Replace with your actual test directory path
# Get the class names (subdirectories) in the train directory
train_class_names = [class_name for class_name in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, class_name))]
# Get the class names (subdirectories) in the test directory
test_class_names = [class_name for class_name in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, class_name))]
# Print the class names in the train and test directories
print("Class names in the train directory:")
print(train_class_names)
print("\nClass names in the test directory:")
print(test_class_names)


Class names in the train directory:
['Kidney_normal', 'kidney_tumor']

Class names in the test directory:
['Kidney_normal', 'kidney_tumor']


In [6]:
image_size = (224, 224)  # Adjust as needed
num_classes = 2  # Change to the number of classes in your dataset
batch_size = 32
learning_rate = 0.01
momentum = 0.9

In [7]:
from keras.preprocessing.image import ImageDataGenerator
# Load training and testing data using data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use a portion of the data for validation
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5270 images belonging to 2 classes.
Found 1316 images belonging to 2 classes.
Found 6586 images belonging to 2 classes.


In [8]:
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
def Inception_block(X, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4):
    # Branch 1
    branch1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(X)
    # Branch 2
    branch2 = Conv2D(f2_conv1, (1, 1), padding='same', activation='relu')(X)
    branch2 = Conv2D(f2_conv3, (3, 3), padding='same', activation='relu')(branch2)
    # Branch 3
    branch3 = Conv2D(f3_conv1, (1, 1), padding='same', activation='relu')(X)
    branch3 = Conv2D(f3_conv5, (5, 5), padding='same', activation='relu')(branch3)
    # Branch 4
    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(X)
    branch4 = Conv2D(f4, (1, 1), padding='same', activation='relu')(branch4)
    # Concatenate the branches
    return Concatenate(axis=-1)([branch1, branch2, branch3, branch4])

In [9]:
def GoogLeNet(input_shape=(224, 224, 3), num_classes=4):
    input_layer = Input(shape=input_shape)
    # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=2, padding='valid', activation='relu')(input_layer)
    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size=(3, 3), strides=2)(X)
    # convolutional layer: filters = 64, strides = 1
    X = Conv2D(filters=64, kernel_size=(1, 1), strides=1, padding='same', activation='relu')(X)
    # convolutional layer: filters = 192, kernel_size = (3,3)
    X = Conv2D(filters=192, kernel_size=(3, 3), padding='same', activation='relu')(X)
    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size=(3, 3), strides=2)(X)
    # 1st Inception block
    X = Inception_block(X, f1=64, f2_conv1=96, f2_conv3=128, f3_conv1=16, f3_conv5=32, f4=32)
    # 2nd Inception block
    X = Inception_block(X, f1=128, f2_conv1=128, f2_conv3=192, f3_conv1=32, f3_conv5=96, f4=64)
    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size=(3, 3), strides=2)(X)
    # 3rd Inception block
    X = Inception_block(X, f1=192, f2_conv1=96, f2_conv3=208, f3_conv1=16, f3_conv5=48, f4=64)
    # Extra network 1:
    X1 = AveragePooling2D(pool_size=(5, 5), strides=3)(X)
    X1 = Conv2D(filters=128, kernel_size=(1, 1), padding='same', activation='relu')(X1)
    X1 = Flatten()(X1)
    X1 = Dense(1024, activation='relu')(X1)
    X1 = Dropout(0.7)(X1)
    X1 = Dense(4, activation='softmax')(X1)  # Assuming you have 4 classes
    # 4th Inception block
    X = Inception_block(X, f1=160, f2_conv1=112, f2_conv3=224, f3_conv1=24, f3_conv5=64, f4=64)
    # 5th Inception block
    X = Inception_block(X, f1=128, f2_conv1=128, f2_conv3=256, f3_conv1=24, f3_conv5=64, f4=64)
    # 6th Inception block
    X = Inception_block(X, f1=112, f2_conv1=144, f2_conv3=288, f3_conv1=32, f3_conv5=64, f4=64)
    # Extra network 2:
    X2 = AveragePooling2D(pool_size=(5, 5), strides=3)(X)
    X2 = Conv2D(filters=128, kernel_size=(1, 1), padding='same', activation='relu')(X2)
    X2 = Flatten()(X2)
    X2 = Dense(1024, activation='relu')(X2)
    X2 = Dropout(0.7)(X2)
    X2 = Dense(1000, activation='softmax')(X2)
    # 7th Inception block
    X = Inception_block(X, f1=256, f2_conv1=160, f2_conv3=320, f3_conv1=32, f3_conv5=128, f4=128)
    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size=(3, 3), strides=2)(X)
    # 8th Inception block
    X = Inception_block(X, f1=256, f2_conv1=160, f2_conv3=320, f3_conv1=32, f3_conv5=128, f4=128)
    # 9th Inception block
    X = Inception_block(X, f1=384, f2_conv1=192, f2_conv3=384, f3_conv1=48, f3_conv5=128, f4=128)
    # Global Average pooling layer
    X = GlobalAveragePooling2D(name='GAPL')(X)
    # Fully connected output layer
    output_layer = Dense(num_classes, activation='softmax')(X)
    return Model(inputs=input_layer, outputs=output_layer)
# Create the GoogLeNet model
model = GoogLeNet(input_shape=(224, 224, 3), num_classes=2)  # Assuming you have 4 classes

In [10]:
# Create the GoogLeNet model
model = GoogLeNet(input_shape=(224, 224, 3), num_classes=num_classes)  # Update the number of classes
# Compile the model using the legacy SGD optimizer and 'categorical_crossentropy' loss
sgd = SGD(learning_rate=learning_rate, decay=1e-6, momentum=momentum, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(train_generator, epochs=50, validation_data=validation_generator)
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Epoch 1/50
165/165 [==============================] - 1040s 6s/step - loss: 0.6887 - accuracy: 0.5402 - val_loss: 0.6856 - val_accuracy: 0.5129
Epoch 2/50
165/165 [==============================] - 111s 673ms/step - loss: 0.5964 - accuracy: 0.6901 - val_loss: 0.4818 - val_accuracy: 0.7774
Epoch 3/50
165/165 [==============================] - 114s 690ms/step - loss: 0.4583 - accuracy: 0.8046 - val_loss: 0.4471 - val_accuracy: 0.8047
Epoch 4/50
165/165 [==============================] - 108s 657ms/step - loss: 0.3885 - accuracy: 0.8455 - val_loss: 0.4535 - val_accuracy: 0.7948
Epoch 5/50
165/165 [==============================] - 130s 791ms/step - loss: 0.2731 - accuracy: 0.8941 - val_loss: 0.4393 - val_accuracy: 0.8245
Epoch 6/50
165/165 [==============================] - 128s 775ms/step - loss: 0.2744 - accuracy: 0.8934 - val_loss: 0.4921 - val_accuracy: 0.8146
Epoch 7/50
165/165 [==============================] - 110s 670ms/step - loss: 0.1875 - accuracy: 0.9249 - val_loss: 0.5282 - v